In [1]:
# from pymoo.algorithms.soo.nonconvex.ga import GA
# from pymoo.optimize import minimize
# from pymoo.core.problem import ElementwiseProblem
# class MyProblem(ElementwiseProblem):
#     def __init__(self):
#         super().__init__(n_var=2, n_obj=1, xl=0, xu=5)

#     def _evaluate(self, x, out, *args, **kwargs):
#         f1 = (x[0] - 2)**2 + (x[1] - 3)**2
#         out["F"] = f1

# problem = MyProblem()
# algorithm = GA(pop_size=50)
# res = minimize(problem,
#                algorithm,
#                ('n_gen', 40),
#                verbose=False)
# print("Best solution found: \nX = %s\nF = %s" % (res.X, res.F))

Evaluate using NSGA

In [2]:
# from pymoo.algorithms.moo.nsga2 import NSGA2
# from pymoo.optimize import minimize
# from pymoo.core.problem import ElementwiseProblem

In [3]:
# # Define the custom problem
# class MyProblem(ElementwiseProblem):
#     def __init__(self):
#         super().__init__(n_var=2, n_obj=2, xl=-10, xu=10)

#     def _evaluate(self, x, out, *args, **kwargs):
#         f1 = (x[0]**2 + x[0] * x[1] - 6)**2
#         f2 = (x[0]**2 + x[1]**3 + 2 * x[0] * x[1] - 3)**2
#         out["F"] = [f1, f2]

# problem = MyProblem()

In [4]:
# # Choose NSGA-II as the method for the multi-objective problem
# algorithm = NSGA2(pop_size=100)

# # Perform the optimization
# res = minimize(problem,
#                algorithm,
#                ('n_gen', 100),
#                verbose=True)

In [5]:
# # Print the best solution(s)
# print('Best solution(s):')
# for solution in res.X:
#     print(f"X = {solution}, F = {problem.evaluate(solution)}")

Evaluate using GA

In [6]:
# from pymoo.algorithms.soo.nonconvex.ga import GA
# from pymoo.optimize import minimize
# from pymoo.core.problem import ElementwiseProblem
# from pymoo.termination.ftol import SingleObjectiveSpaceTermination
# from pymoo.termination.robust import RobustTermination
# from pymoo.termination.default import DefaultSingleObjectiveTermination
# from pymoo.operators.mutation.pm import PolynomialMutation

In [7]:
# class Mastorakis1Problem(ElementwiseProblem):
#     def __init__(self):
#         super().__init__(n_var=2,n_obj=1,xl=-10,xu=10)
#     def _evaluate(self, x, out, *args, **kwargs):
#         f1 = (x[0]**2+x[0]*x[1]-6)**2+(x[0]**2+x[1]**3+2*x[0]*x[1]**2-3)**2
#         out['F'] = f1

# m1problem = Mastorakis1Problem()

In [8]:
# # GA parameter
# mutation = PolynomialMutation(prob=0.2)
# algorithmGA = GA(pop_size=40,eliminate_duplicates=True,mutation=mutation)

# # termination variable
# termination1 = RobustTermination(
#     SingleObjectiveSpaceTermination(tol=0.005, n_skip=5), period=20)
# termination2 = DefaultSingleObjectiveTermination(
#     xtol=1e-8,
#     cvtol=1e-6,
#     ftol=1e-6,
#     period=20,
#     n_max_gen=100,
#     n_max_evals=100000
# )

# m1res = minimize(problem=m1problem,
#                  algorithm=algorithmGA,
#                  termination=termination2,
#                  seed=None,
#                  verbose=True)

# print("Best solution found: \nX = %s\nF = %s" % (m1res.X, m1res.F))

# Root GA

Problem 1 Pak Kun

In [9]:
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.optimize import minimize
from pymoo.core.problem import ElementwiseProblem
from pymoo.termination.default import DefaultSingleObjectiveTermination
from pymoo.operators.mutation.pm import PolynomialMutation
import numpy as np
from itertools import product,combinations
import matplotlib.pyplot as plt

In [10]:
m_cluster = 50
gamma = 0.2
epsilon = 10**(-3)
delta = 0.01
k_cluster = 10
n_point = 100
k_max = 300
dim = 2
epsilon = 10**(-5)
p_mutation = 0.1
num_bits = 64  # Number of bits for each number
boundaries = np.array([(-10,10) for _ in range (dim)])

In [11]:
def objective_function(x):
    f1 = np.exp(x[0]-x[1])-np.sin(x[0]+x[1])
    f2 = (x[0]*x[1])**2-np.cos(x[0]+x[1])
    # f_list = [f1,f2]
    # denom = 0
    # for f in f_list:
    #     denom +=np.abs(f)
    # F = 1/(1+denom)
    return np.array([f1,f2])


In [12]:
def slice_hypercube(lower_bounds, upper_bounds, interval, dim):
    # Create a list of arrays, each containing points spaced h apart for each dimension
    points = [np.arange(lower_bounds[i], upper_bounds[i], interval) for i in range(dim)]
    
    # Use meshgrid to create a grid of points in n-dimensional space
    grids = np.meshgrid(*points, indexing='ij')
    
    # Flatten and combine the grid points into a single 2D array
    grid_points = np.vstack([grid.ravel() for grid in grids]).T
    
    # Generate all vertices for smaller hypercubes within each grid cell
    return np.array([grid_points + offset for offset in product([0, interval], repeat=dim)])

In [13]:
# how many parts/slices do you desire in each dimension?
parts = 100

# length of each parts in each dimension
inc_int = (boundaries[0,1]-boundaries[0,0])/parts

# Hypercubes
hypercubes_edges = slice_hypercube(lower_bounds=boundaries[:,0],
                    upper_bounds=boundaries[:,1],
                    interval=inc_int,
                    dim=dim)

In [14]:
cluster = []
for hypercube_id in range(hypercubes_edges.shape[1]):
    X0 = hypercubes_edges[:,hypercube_id,:]
    F_list = objective_function(X0.T)

    # cek jika f yang berubah tanda dari F_list jika dievaluasi di tiap edge hypercube
    product_combination = np.array([[a*b for a,b in combinations(F_list[i],2)] for i in range (F_list.shape[0])])

    # jika semua f dari F_list berubah tanda jika dievaluasi di tiap edge hypercube, maka ada akar di situ
    change_sign = np.array([np.any(product_combination[i]<0) for i in range (product_combination.shape[0])])
    if np.all(change_sign==True):
        # print(f'Ada akar di sini: \nX0={X0}')
        cluster.append(X0)

cluster = np.array(cluster)
cluster.shape[0]

12

In [15]:
# GA parameter
mutation = PolynomialMutation(prob=0.1)
algorithmGAp1 = GA(pop_size=n_point,eliminate_duplicates=True,mutation=mutation)

# termination variable
terminationp1 = DefaultSingleObjectiveTermination(
    xtol=1e-8,
    cvtol=1e-6,
    ftol=1e-6,
    period=20,
    n_max_gen=k_max,
    n_max_evals=100000
)

def root_ga(n_var,n_obj,xl,xu,objective_function,algorithmGA,termination,seed=1,verbose=False):
    class PakKun1(ElementwiseProblem):
        def __init__(self):
            super().__init__(n_var=n_var,n_obj=n_obj,xl=xl,xu=xu)
        def _evaluate(self, x, out, *args, **kwargs):
            F_array = objective_function(x)
            denom = 0
            for f in F_array:
                denom +=np.abs(f)
            F = 1/(1+denom)
            out['F'] = -F

    problem = PakKun1()
    result = minimize(problem=problem,
                    algorithm=algorithmGA,
                    termination=termination,
                    seed=seed,
                    verbose=verbose)
    return [result.X,result.F]

roots = []
values = []
for i in range (cluster.shape[0]):
    root,value = root_ga(2,1,xl=cluster[i,:,:][0],xu=cluster[i,:,:][-1],objective_function=objective_function,
                        algorithmGA=algorithmGAp1,termination=terminationp1,seed=0)
    roots.append(root)
    values.append(value)
roots = np.array(roots)
values = np.array(values)
print(roots,"\n\n",values)

[[-6.43712573  0.15534876]
 [-6.2        -0.16104581]
 [-6.4         0.15618943]
 [-6.1170816  -0.16348884]
 [-1.          1.        ]
 [-0.93179264  1.06820304]
 [-0.15616774  6.4       ]
 [-0.15529     6.43981113]
 [ 0.16333465  6.12243972]
 [ 0.1610043   6.2       ]
 [ 0.59992547  0.76093895]
 [ 0.66761908  0.68959885]] 

 [[-0.99996231]
 [-0.9257837 ]
 [-0.96344378]
 [-0.99982906]
 [-0.88079708]
 [-0.99934717]
 [-0.96057541]
 [-0.99989141]
 [-0.99997673]
 [-0.92947715]
 [-0.88749292]
 [-0.99901963]]
